In [1]:
import numpy as np
import csv
import pickle
import matplotlib
import math
from itertools import combinations
import itertools
import pandas as pd
import matplotlib
# from pyvis.network import Network
# import networkx as nx
# from networkx.algorithms import bipartite
# from cdlib import algorithms
# from cdlib import NodeClustering
# import altair as alt
# import sys
# import os
# sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
# if True:  # In order to bypass isort when saving
#     import altairThemes

spot_diameter = 89.43 #pixels

In [2]:
def readCsv(x):
  """Parse file."""
  #colNames = ["method", "benchmark", "start", "end", "time", "memory"]
  df = pd.read_csv(x, sep=",")

  return df

def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df

In [13]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)

# Load connected_components (assume already have it)
with open("/Users/victoriagao/local_docs/NEST/stored_variables/filtered_connected_components.pkl", 'rb') as file:
    filtered_connected_components = pickle.load(file)

# Load Deisha's subtype label
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    subtype_label_file='/Users/victoriagao/local_docs/schwartz_data/PDAC_64630_subtype.csv' 
    subtype_label=[]
    with open(subtype_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            subtype_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(subtype_label)):
        barcode_type[subtype_label[i][0]] = subtype_label[i][1]

barcode_type_df = pd.Series(barcode_type).rename('subtype')

# Load NEST output file into a 2D array
# filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
# inputFile = '/Users/victoriagao/local_docs/NEST/output/From_Fatema/'+filename_str
# df = pd.read_csv(inputFile, sep=",")
# csv_record_final = df.values.tolist()
# df_column_names = list(df.columns)
# csv_record_final = [df_column_names] + csv_record_final

# Load NEST output 
df = readCsv("/Users/victoriagao/local_docs/NEST/output/From_Fatema/NEST_combined_output_PDAC_64630.csv")
csv_record_df = preprocessDf(df)


In [12]:
barcode_type_df

AAACCGGGTAGGTACC-1      BasalB
AAACCGTTCGTCCAGG-1       Mixed
AAACGAGACGGTTGAT-1       Mixed
AAACTGCTGGCTCCAA-1       Mixed
AAAGACCCAAGTCGCG-1      BasalB
                        ...   
TTGTTCAGTGTGCTAC-1       Mixed
TTGTTGTGTGTCAAGA-1       Mixed
TTGTTTCACATCCAGG-1       Mixed
TTGTTTCCATACAACT-1      BasalB
TTGTTTGTGTAAATTC-1    ClassicB
Name: subtype, Length: 1406, dtype: object

In [15]:
# Map subtypes to cells in ligand-receptor data
csv_record_df['from_subtype'] = csv_record_df['from_cell'].map(barcode_type_df)
csv_record_df['to_subtype'] = csv_record_df['to_cell'].map(barcode_type_df)


In [19]:
# Filter pairs where from and to subtype are the same
same_subtype_df = csv_record_df[csv_record_df['from_subtype'] == csv_record_df['to_subtype']]
same_subtype_df

,from_cell,to_cell,ligand,receptor,attention_score,component,from_id,to_id,ligand-receptor,from_subtype,to_subtype
0,ACGCGCTACACAGGGT-1,AAACCGGGTAGGTACC-1,LGALS3,NPTN,0.911919,2,132,0,LGALS3-NPTN,BasalB,BasalB
4,CCACAGTACCCATCCT-1,AACCGCTAAGGGATGC-1,FN1,RPSA,0.929303,16,425,23,FN1-RPSA,Mixed,Mixed
5,AATAACACTAGAACAA-1,AACTCTCAGTGTGCTC-1,INS,HLA,0.967800,19,62,35,INS-HLA,Mixed,Mixed
6,GAATTTCTCGCTGCAG-1,AACTTGCCCGTATGCA-1,FN1,RPSA,0.929611,14,696,36,FN1-RPSA,BasalB,BasalB
7,GCGATTGTTAACGTTA-1,AAGAGCTCTTTATCGG-1,PTPRF,RACK1,0.911204,20,802,39,PTPRF-RACK1,Mixed,Mixed
...,...,...,...,...,...,...,...,...,...,...,...
1087,AACGTCAGACTAGTGG-1,TTGGCTCGCATGAGAC-1,TGFB1,ITGB5,0.834446,17,31,1389,TGFB1-ITGB5,BasalB,BasalB
1088,AACGTCAGACTAGTGG-1,TTGGCTCGCATGAGAC-1,TGFB1,EGFR,0.839991,17,31,1389,TGFB1-EGFR,BasalB,BasalB
1089,AGATTATAGGACGTTT-1,TTGTAATCCGTACTCG-1,TGFB1,ITGB5,0.829314,9,184,1394,TGFB1-ITGB5,BasalB,BasalB
1090,AGATTATAGGACGTTT-1,TTGTAATCCGTACTCG-1,TGFB1,SDC2,0.855152,9,184,1394,TGFB1-SDC2,BasalB,BasalB


In [20]:
# Create unique ligand-receptor pairs
same_subtype_df['ligand_receptor'] = same_subtype_df['ligand'] + '-' + same_subtype_df['receptor']

# Create count matrix
count_matrix = same_subtype_df.groupby(['from_subtype', 'ligand_receptor']).size().unstack(fill_value=0)

count_matrix

/var/folders/b0/dbz12pb13hg1zrjq45vwhc680000gn/T/ipykernel_6421/914123905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  same_subtype_df['ligand_receptor'] = same_subtype_df['ligand'] + '-' + same_subtype_df['receptor']


ligand_receptor,A2M-TNFRSF14,ADAM9-ITGA6,AIMP1-RACK1,APOE-SDC1,APOE-SDC4,APP-SDC1,ARF1-LRP5,BMP2-ACVR1,BMP2-ENG,BMP4-BMPR2,...,THBS2-CD47,TIMP1-LRP1,TNC-ITGB6,TNC-SDC1,TNFSF13-TNFRSF14,TNFSF13-TNFRSF1A,TTR-DDR1,TTR-STRA6,WNT5A-TFRC,no-ligand-no-receptor
from_subtype,,,,,,,,,,,,,,,,,,,,,
BasalB,0,2,0,0,1,0,0,0,0,1,...,3,1,0,0,0,0,1,0,0,1
ClassicA,1,0,0,0,0,1,3,1,1,0,...,1,2,2,1,1,2,0,0,0,0
ClassicB,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,1,0,0
Mixed,0,0,1,6,6,0,0,0,0,0,...,0,11,0,0,0,0,5,0,1,0


In [4]:
# Get a list of unique LR pairs from the csv file
LR_pairs = df_processed['ligand-receptor'].unique()
len(LR_pairs)

147

In [5]:
barcode_type

{'AAACCGGGTAGGTACC-1': 'BasalB',
 'AAACCGTTCGTCCAGG-1': 'Mixed',
 'AAACGAGACGGTTGAT-1': 'Mixed',
 'AAACTGCTGGCTCCAA-1': 'Mixed',
 'AAAGACCCAAGTCGCG-1': 'BasalB',
 'AAAGGCTCTCGCGCCG-1': 'ClassicB',
 'AAAGGGATGTAGCAAG-1': 'Mixed',
 'AAAGGGCAGCTTGAAT-1': 'ClassicB',
 'AAAGTAGCATTGCTCA-1': 'Mixed',
 'AAAGTGTGATTTATCT-1': 'BasalB',
 'AAATACCTATAAGCAT-1': 'Mixed',
 'AAATAGGGTGCTATTG-1': 'Mixed',
 'AAATCGTGTACCACAA-1': 'BasalB',
 'AAATGGTCAATGTGCC-1': 'BasalB',
 'AAATTAACGGGTAGCT-1': 'Mixed',
 'AAATTGATAGTCCTTT-1': 'Mixed',
 'AAATTTGCGGGTGTGG-1': 'Mixed',
 'AACAACTGGTAGTTGC-1': 'BasalB',
 'AACAATTACTCTACGC-1': 'BasalB',
 'AACACACGCTCGCCGC-1': 'Mixed',
 'AACAGGATGGGCCGCG-1': 'BasalB',
 'AACCAAGACTTCTCTG-1': 'BasalB',
 'AACCCAGAGACGGAGA-1': 'BasalB',
 'AACCGCTAAGGGATGC-1': 'Mixed',
 'AACCTTTAAATACGGT-1': 'ClassicA',
 'AACGATAATGCCGTAG-1': 'Mixed',
 'AACGCGGTCTCCAGCC-1': 'Mixed',
 'AACGCTGTTGCTGAAA-1': 'ClassicA',
 'AACGGACGTACGTATA-1': 'Mixed',
 'AACGGCCATCTCCGGT-1': 'Mixed',
 'AACGTACTGTGGGTAC

{1: [['ACGCGCTACACAGGGT-1',
   'AAACCGGGTAGGTACC-1',
   'LGALS3',
   'NPTN',
   0.9119191408157348,
   2,
   132,
   0],
  ['TTGGACATGTGGCTTA-1',
   'AGCCCATACATGTAAG-1',
   'LGALS3',
   'ITGB4',
   0.9117170333862304,
   15,
   1386,
   192],
  ['AACAATTACTCTACGC-1',
   'ATTCAGTAGCAGGGTC-1',
   'RPS19',
   'RPSA',
   0.9298113584518432,
   12,
   18,
   317],
  ['AACTCAAGTTAATTGC-1',
   'CCTGTAAGACATGATA-1',
   'THBS1',
   'SDC1',
   0.9103351473808288,
   18,
   34,
   499],
  ['TCCCGGGTGTGCTGCT-1',
   'TACGATGTTGATCATC-1',
   'PTPRF',
   'RACK1',
   0.9114265680313112,
   17,
   1157,
   1061],
  ['TCCCGGGTGTGCTGCT-1',
   'TGTAGTGATCTATAAT-1',
   'PTPRF',
   'RACK1',
   0.9121753334999084,
   17,
   1157,
   1293],
  ['CGGTGAAGACTAAAGT-1',
   'AACCAAGACTTCTCTG-1',
   'TGFB1',
   'ITGB5',
   0.8489698052406311,
   15,
   563,
   21],
  ['TGCAGAGTACCGAGCA-1',
   'ACGATGCATATGTTAT-1',
   'TGFB1',
   'ITGB5',
   0.8359954595565796,
   18,
   1254,
   128],
  ['AAACCGGGTAGGTACC-1',
   'A

In [7]:
# Create count matrix for all 144 of LR pairs in each region
num_sets = len(matched_records_set_lib)
num_pairs = len(LR_pairs)
count_matrix = pd.DataFrame(index=range(1, num_sets + 1), columns=LR_pairs)

# Fill the count matrix
for set_number, records in matched_records_set_lib.items():
    for lr_pair in LR_pairs:
        count = sum(1 for record in records if f"{record[2]}-{record[3]}" == lr_pair)
        count_matrix.loc[set_number, lr_pair] = count

# # Print the count matrix
# print(count_matrix)

count_matrix_df = pd.DataFrame(count_matrix)

   LGALS3-NPTN FN1-RPSA PTPRF-RACK1 INS-HLA TIMP1-LRP1 THBS2-LRP1 THBS1-SDC1  \
1            2        0           2       0          0          0          1   
2            1        0           0       0          0          0          0   
3            6       55          27       2         21          2         18   
4            2        0           0       3          0          0          2   
5            2        0           7       0          2          0          0   
6            0        0           0       0          0          0          0   
7            1        0           0       0          0          0          0   
8            0        0           0       0          0          0          0   
9            0        0           0       0          0          0          0   
10           0        0           0       0          0          0          0   
11           0        0           0       0          0          0          0   
12           0        0           0     

In [33]:
# Compiling all log odds calculationg into one big sheet
number_of_regions = len(filtered_connected_components)

# interested_region = 0
# comparing_region = 6

# Define a folder where you want to save the tables
output_folder = "/Users/victoriagao/local_docs/NEST/output/"

epsilon = 1e-10 

regions = count_matrix_df.columns.tolist()

# Create all possible combinations of 2 regions
region_combinations = itertools.combinations(regions, 2)
whole_log_odds_df = pd.DataFrame(LR_pairs, columns=["L-R Pair"])

for i in range(1,number_of_regions):
    print("region",i)
    for j in range(i + 1, number_of_regions):
        print("comparing with region",j)
        # region1 = regions[i]
        # region2 = regions[j]
        single_log_odds_df = pd.DataFrame()
        result_data = []
       
        # Select rows from count_matrix_df for the two regions
        region1_data = count_matrix_df.loc[i]
        region2_data = count_matrix_df.loc[j]

        # print(region1_data)

        for lr_pair in LR_pairs:
            
            
            # Create a DataFrame to store the contingency table
            columns = ['Region 1', 'Region 2']
            index = ['# of LR pair', '# of non LR pair']
            contingency_table = pd.DataFrame(index=index, columns=columns)
            contingency_table.fillna(0, inplace=True)  # Fill NaN cells with 0
            
            # # Calculate the sum of LR pairs other than lr_pair
            count_matrix_interested_row = count_matrix_df.iloc[i]
            count_matrix_comparing_row = count_matrix_df.iloc[j]
            
            # # Populate the contingency table
            contingency_table.loc['# of LR pair','Region 1'] = count_matrix_interested_row[lr_pair]
            contingency_table.loc['# of non LR pair','Region 1'] = count_matrix_interested_row.drop(lr_pair).sum()
            
            contingency_table.loc['# of LR pair','Region 2'] = count_matrix_comparing_row[lr_pair]
            contingency_table.loc['# of non LR pair','Region 2'] = count_matrix_comparing_row.drop(lr_pair).sum()

            # Extract the relevant counts from the contingency table
            Ad = contingency_table.loc['# of LR pair','Region 1']
            Bc = contingency_table.loc['# of LR pair','Region 2']
            Bd = contingency_table.loc['# of non LR pair','Region 1']
            Ac = contingency_table.loc['# of non LR pair','Region 2']

            # Handle corner cases
            if Ad == 0 and Bc != 0:
                log_odds_ratio = -float('inf')  # Negative infinity
            elif Bc == 0 and Ad != 0:
                log_odds_ratio = float('inf')   # Positive infinity
            elif Bc == 0 and Ad == 0: # Cornor case: if the LR is not observed in neither region
                log_odds_ratio = None
            else:
                # odds_ratio = (Ad / Bc) / (Bd / Ac)
                # log_odds_ratio = np.log(odds_ratio)
                Bc = max(Bc, epsilon)
                Bd = max(Bd, epsilon)
                Ac = max(Ac, epsilon)
                odds_ratio = (Ad / Bc) / (Bd / Ac)
            
                if odds_ratio == 0:  # To handle the case where odds_ratio is very close to 0
                    log_odds_ratio = -float('inf')
                else:
                    log_odds_ratio = np.log(odds_ratio)
            
        # Store the result in the result_data list
            result_data.append({'L-R Pair': lr_pair, f"region{i} VS {j}": log_odds_ratio})
            
     # Create a DataFrame from the result_data list
        single_log_odds_df = pd.DataFrame(result_data)
        whole_log_odds_df = pd.merge(whole_log_odds_df, single_log_odds_df, on='L-R Pair', how='inner')

        # log_odds_df = log_odds_df.sort_values(by='log_odds_ratio', ascending=True)
        # log_odds_df['Ranking'] = log_odds_df['log_odds_ratio'].rank(ascending=False)

        # Save
        file_name = f"BIG_log_odds.csv"
        table_path = os.path.join(output_folder, file_name)
        whole_log_odds_df.to_csv(table_path, index=False)

# # Create a DataFrame from the region_results list
# result_df = pd.DataFrame(region_results)

# # Print the results
# print(result_df)

range 1
comparing with range 2
results length 147
length of BIG df 147
comparing with range 3
results length 147
length of BIG df 147
comparing with range 4
results length 147
length of BIG df 147
comparing with range 5
results length 147
length of BIG df 147
comparing with range 6
results length 147
length of BIG df 147
comparing with range 7
results length 147
length of BIG df 147
comparing with range 8
results length 147
length of BIG df 147
comparing with range 9
results length 147
length of BIG df 147
comparing with range 10
results length 147
length of BIG df 147
comparing with range 11
results length 147
length of BIG df 147
comparing with range 12
results length 147
length of BIG df 147
comparing with range 13
results length 147
length of BIG df 147
comparing with range 14
results length 147
length of BIG df 147
comparing with range 15
results length 147
length of BIG df 147
comparing with range 16
results length 147
length of BIG df 147
comparing with range 17
results length 1